In [1]:
%matplotlib inline

import numpy
import matplotlib.pyplot as plt
from astropy import units
from astropy import constants as const

# Optional: 
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('svg')

In [3]:
def g_LS(L, S, J):
    """
    Calculates Landé factor from LS coupling.
    """
    if J == 0:
        return 1
    else:
        return 1 + 0.5 * (J * (J + 1) + S * (S + 1) - L * (L + 1)) / (J * (J + 1))


def g_eff(g1, g2, J1, J2):
    """
    Calculates the effective Landé factor.
    """
    d = J1 * (J1 + 1) - J2 * (J2 + 1)
    return 0.5 * (g1 + g2) + 0.25 * (g1 - g2) * d


def parse_term(multiplicity, letter, J):
    """
    Parses term symbol in the form ^2S+1 L _J., 
    where multiplicity is 2S + 1, letter is the letter of the term (SPDF...)
    and J is J.
    """
    S = (multiplicity - 1) / 2
    terms = {key: index for index, key in enumerate('SPDFGHIJK')}
    L = terms[letter.upper()]
    return (L, S, J)

def zeeman_strength(J_l, J_u, M_l, M_u):
    """
    Calculates strengths of Zeeman components.
    """
    J, M = J_l, M_l
    if M_u - M_l == 1:
        if J_u - J_l == 1:
            strength = (3 * (J + M + 1) * (J + M + 2)) / (2 * (J + 1) * (2 * J + 1) * (2 * J + 3))
        elif J_u - J_l == 0:
            strength = (3 * (J - M) * (J + M + 1)) / (2 * J * (J + 1) * (2 * J + 1))
        elif J_u - J_l == -1:
            strength = (3 * (J - M) * (J - M - 1)) / (2 * J * (2 * J - 1) * (2 * J + 1))
        else:
            raise ValueError('Invalid transition, J_u - J_l != -1, 0, 1')
    elif M_u - M_l == 0:
        if J_u - J_l == 1:
            strength = (3 * (J - M + 1) * (J + M + 1)) / ((J + 1) * (2 * J + 1) * (2 * J + 3))
        elif J_u - J_l == 0:
            strength = 3 * M**2 / (J * (J + 1) * (2 * J + 1))
        elif J_u - J_l == -1:
            strength = (3 * (J - M) * (J + M)) / (J * (2 * J - 1) * (2 * J + 1))
        else:
            raise ValueError('Invalid transition, J_u - J_l != -1, 0, 1')
    elif M_u - M_l == -1:
        if J_u - J_l == 1:
            strength = (3 * (J - M + 1) * (J - M + 2)) / (2 * (J + 1) * (2 * J + 1) * (2 * J + 3))
        elif J_u - J_l == 0:
            strength = (3 * (J + M) * (J - M + 1)) / (2 * J * (J + 1) * (2 * J + 1))
        elif J_u - J_l == -1:
            strength = (3 * (J + M) * (J + M - 1)) / (2 * J * (2 * J - 1) * (2 * J + 1))
        else:
            raise ValueError('Invalid transition, J_u - J_l != -1, 0, 1')
    else:
        raise ValueError('Invalid transition, M_u - M_l != -1, 0, 1')
    return strength


def zeeman_broadening(wave_0, B):
    return (const.e.si * wave_0**2 * B / (4 * numpy.pi * const.m_e * const.c)).to('nm')


def zeeman_components(upper_term, lower_term):
    """
    Calculates energy separation of different Zeeman
    components (in units of \mu_B * B),
    and the strength of Zeeman components.
    """
    L_u, S_u, J_u = parse_term(*upper_term)
    L_l, S_l, J_l = parse_term(*lower_term)
    # Find possible combinations
    if J_u == 0:
        M_u = numpy.array([0.])
    else:
        M_u = numpy.arange(-J_u, J_u + 1)
    if J_l == 0:
        M_l = numpy.array([0.])
    else:
        M_l = numpy.arange(-J_l, J_l + 1)
    g_u = g_LS(L_u, S_u, J_u)
    g_l = g_LS(L_l, S_l, J_l)
    M_diff = M_u[:, numpy.newaxis] - M_l[numpy.newaxis, :]
    permitted = numpy.abs(M_diff) <= 1
    energy_diff = g_u * M_u[:, numpy.newaxis] - g_l * M_l[numpy.newaxis, :]
    # redefine in matrix form
    M_u, M_l = M_diff + M_l[numpy.newaxis, :], M_u[:, numpy.newaxis] - M_diff
    result = {}
    for q, m in zip(['sigma_r', 'pi', 'sigma_b'], [-1, 0, 1]):
        q_index = permitted & (M_diff == m)
        # Get only unique energies for a given Mj
        _, unique = numpy.unique(energy_diff[q_index], return_index=True)
        Mu = M_u[q_index][unique]
        Ml = M_l[q_index][unique]
        strength = numpy.array([zeeman_strength(J_l, J_u, ml, mu) for ml, mu in zip(Ml, Mu)])
        result[q] = {'energy': energy_diff[q_index][unique], 'strength': strength}
    return result


def plot_zeeman(upper_term, lower_term, plot_geff=False):   
    """
    Plots Zeeman components in energy vs strength following
    the usual convention that \pi are up and \sigma are down.
    """
    fig, ax = plt.subplots()
    cfg = {'sigma_r': ['r-', -1], 'pi': ['k-', 1], 'sigma_b': ['b-', -1]}
    for component, value in zeeman_components(upper_term, lower_term).items():
        for e, s in zip(value['energy'], value['strength']):
            ax.plot([e, e], [0, s * cfg[component][1]], cfg[component][0])
    ax.axhline(y=0, color='k')
    if plot_geff:
        L_u, S_u, J_u = parse_term(*upper_term)
        L_l, S_l, J_l = parse_term(*lower_term)
        g_u = g_LS(L_u, S_u, J_u)
        g_l = g_LS(L_l, S_l, J_l)
        geff = g_eff(g_l, g_u, J_l, J_u)
        ax.axvline(x=geff, color='y', ls='--')
        ax.axvline(x=-geff, color='y', ls='--')
    ax.axis('off')